In [1]:
from config import pg_name, pg_user, pg_host, pg_port
from cred import pg_password
import psycopg2
conn = psycopg2.connect(dbname=pg_name, user =pg_user, host=pg_host, password=pg_password, port =pg_port)
cur = conn.cursor()

Input Below

In [ ]:
date = '2024-10-02'

In [2]:
parse_sql = """
call facebook_staging.key_standard();
call facebook_staging.fact_daily_standard();
call facebook_staging.fact_conversion();
"""
cur.execute(parse_sql)
conn.commit()

In [3]:
job_mgr_check = """
select report_status, count(*) from facebook_staging.job_manager group by 1;
"""
counter = 0

cur.execute(job_mgr_check)
for i in cur.fetchall():
    print(i)
    counter+= 1


parse_mgr_check = """
select staging_status, count(*) from facebook_staging.query_data group by 1;
"""
cur.execute(parse_mgr_check)
for i in cur.fetchall():
    print(i)
    counter+= 1

('Job Processed', 224)
('parsed', 487)


In [ ]:
if counter != 2:
    print(hello)
    

In [ ]:
# update_check_timing_sql = f"""
# update facebook.accounts
# set
# 	last_updated_key = '{date} 00:00:00',
# 	last_update_fact = '{date} 00:00:00'
# from (select cast(account_id as bigint) as account_id from facebook_staging.query_data group by 1) as a
# where facebook.accounts.id = a.account_id;
# """
# if counter == 2:
#     cur.execute(update_check_timing_sql)
#     conn.commit()
# else:
#     print(hello)

In [4]:
update_prod_tables = """
call facebook_staging.clear_queues();
call facebook_prod.napa_standard();
call facebook_prod.jameshardie_standard();
call facebook_prod.aflac_standard();
call facebook_prod.humana_pharm_standard();
call facebook_prod.xumo_daily_standard()
"""
cur.execute(update_prod_tables)
conn.commit()

In [5]:
update_check_timing_sql = """
update facebook.accounts
set last_updated_key = update_date,
	last_update_fact = update_date

from (
	select account_id, 
	cast(max(date) as timestamp) as update_date 
	from facebook.fact_daily_standard group by account_id) as fds
where id = fds.account_id
"""
cur.execute(update_check_timing_sql)
conn.commit()